Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Travelling salesman problem

In [11]:
import pandas as pd
import numpy as np
import itertools
import sys
from geopy.distance import geodesic
from tqdm import tqdm
import random

## Loading csv datas

In [12]:
CITIES = pd.read_csv('china.csv', header=None, names=['name', 'lat', 'lon'])

In [ ]:
# Create the distance matrix using geodesic distances
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in itertools.combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


## Greedy nearest neighbor with evoulutionary inver over crossover

In [ ]:

# Greedy heuristic to generate initial population
def greedy_heuristic(dists, population_size):
    def nearest_neighbor(start):
        n = len(dists)
        visited = [False] * n
        path = [start]
        visited[start] = True
        for _ in range(n - 1):
            last = path[-1]
            nearest = None
            nearest_dist = float('inf')
            for i in range(n):
                if not visited[i] and dists[last][i] < nearest_dist:
                    nearest = i
                    nearest_dist = dists[last][i]
            path.append(nearest)
            visited[nearest] = True
        path.append(start)
        return path

    population = []
    for _ in range(population_size // 2):
        start = random.randint(0, len(dists) - 1)
        path = nearest_neighbor(start)
        population.append(path)

    # Add random paths to increase diversity
    for _ in range(population_size // 2):
        path = list(range(1, len(dists)))
        random.shuffle(path)
        path = [0] + path + [0]
        population.append(path)

    return population

# Local search to improve individuals
def local_search(path, dists):
    def calculate_cost(path):
        return sum(dists[path[i], path[i + 1]] for i in range(len(path) - 1))

    best_cost = calculate_cost(path)
    best_path = path[:]
    for i in range(1, len(path) - 2):
        for j in range(i + 1, len(path) - 1):
            new_path = path[:]
            new_path[i:j+1] = reversed(new_path[i:j+1])
            new_cost = calculate_cost(new_path)
            if new_cost < best_cost:
                best_cost = new_cost
                best_path = new_path
    return best_path

# Evolutionary Algorithm for TSP with improvements
def evolutionary_algorithm(dists, population_size, mutation_rate, generations):
    def calculate_cost(path):
        return sum(dists[path[i], path[i + 1]] for i in range(len(path) - 1))

    def create_initial_population():
        return greedy_heuristic(dists, population_size)

    def inver_over_crossover(parent1, parent2):
        child = parent1[:]
        for i in range(1, len(child) - 1):
            if random.random() < 0.5:
                j = random.choice(range(1, len(child) - 1))
                if random.random() < 0.5:
                    child[i:j+1] = reversed(child[i:j+1])
                else:
                    child[i], child[j] = child[j], child[i]
        return child

    def mutate(path):
        if random.random() < mutation_rate:
            i, j = random.sample(range(1, len(path) - 1), 2)
            path[i], path[j] = path[j], path[i]

    def select_parents(population):
        return random.choices(population, weights=[1 / calculate_cost(p) for p in population], k=2)

    population = create_initial_population()
    best_path = min(population, key=calculate_cost)
    best_cost = calculate_cost(best_path)

    for generation in tqdm(range(generations), desc="Evolutionary Algorithm"):
        new_population = []
        # Elitism: Preserve the best individual
        new_population.append(best_path)
        for _ in range(population_size - 1):
            parent1, parent2 = select_parents(population)
            child = inver_over_crossover(parent1, parent2)
            mutate(child)
            child = local_search(child, dists)  # Apply local search
            new_population.append(child)
        population = new_population

        current_best_path = min(population, key=calculate_cost)
        current_best_cost = calculate_cost(current_best_path)
        if current_best_cost < best_cost:
            best_path = current_best_path
            best_cost = current_best_cost

        # Adaptive mutation rate
        mutation_rate = max(0.01, mutation_rate * 0.99)

    return best_cost, best_path


In [ ]:
population_size = 100
mutation_rate = 0.05
generations = 100

# Solve the TSP using the Evolutionary Algorithm with improvements
opt_cost, opt_path = evolutionary_algorithm(DIST_MATRIX, population_size, mutation_rate, generations)

# Convert the path indices to city names
opt_path_cities = [CITIES['name'][i] for i in opt_path]

print(f"Approximate cost: {opt_cost:.2f} km")
print(f"Approximate path: {opt_path_cities}")

Evolutionary Algorithm:   0%|          | 0/100 [00:00<?, ?it/s]

Evolutionary Algorithm: 100%|██████████| 100/100 [00:58<00:00,  1.71it/s]

Approximate cost: 4436.03 km
Approximate path: ['Rimini', 'Forlì', 'Ravenna', 'Ferrara', 'Bologna', 'Modena', "Reggio nell'Emilia", 'Parma', 'Piacenza', 'Milan', 'Monza', 'Bergamo', 'Brescia', 'Verona', 'Vicenza', 'Padua', 'Venice', 'Trieste', 'Bolzano', 'Trento', 'Novara', 'Turin', 'Genoa', 'Leghorn', 'Prato', 'Florence', 'Perugia', 'Terni', 'Rome', 'Latina', 'Giugliano in Campania', 'Naples', 'Salerno', 'Foggia', 'Andria', 'Bari', 'Taranto', 'Messina', 'Reggio di Calabria', 'Catania', 'Syracuse', 'Palermo', 'Cagliari', 'Sassari', 'Pescara', 'Ancona', 'Rimini']
